<a href="https://colab.research.google.com/github/aks-hit/Book2Life/blob/main/Character_Image_Generation_from_Books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from langchain.vectorstores import Chroma  # Chroma vector store
from langchain.embeddings import CohereEmbeddings  # Cohere embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Recursive character text splitter
from langchain.llms import Cohere  # Cohere LLM
from langchain.chains import VectorDBQA  # VectorDBQA chain
from langchain.document_loaders import PyMuPDFLoader  # PyMuPDF document loaders. Read more: https://pymupdf.readthedocs.io/en/latest/
import os
import io
import warnings
from PIL import Image
from stability_sdk import client  # Stability SDK
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

In [4]:
os.environ["COHERE_API_KEY"] = "Your_COHERE_API_KEY"

os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'  # Host URL should not be prepended with "https" nor should it have a trailing slash.
os.environ["STABILITY_KEY"] = "Your_STABILITY_KEY"

In [43]:
loader = PyMuPDFLoader("/content/The Complete Harry Potter (Harry Potter and the Sorcerer's Stone; Harry Potter and the Chamber of Secrets; Harry Potter and the Prisoner of Azkaban; Harry Potter and the Goblet of Fire; Harry Potter and the Order .pdf")  # Replace with your file path
documents = loader.load()

In [44]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [45]:
embeddings = CohereEmbeddings(cohere_api_key=os.environ["COHERE_API_KEY"])  # Create Cohere embeddings
vectordb = Chroma.from_documents(texts, embeddings)  # Create Chroma vector store

In [46]:
qa = VectorDBQA.from_chain_type(llm=Cohere(cohere_api_key=os.environ["COHERE_API_KEY"]), chain_type="stuff", vectorstore=vectordb)  # Create QA chain

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:256: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [47]:
query = "Can you tell me more about Hermoine style, behaviour and character"  # Feel free to change the query to your own
qa.run(query)

' Hermione is a smart and ambitious student who excels in academics and is passionate about learning. \n\nShe is known for her dedication to studying and completing homework, even while recovering in the hospital wing after a mysterious disappearance. \nHer behavior is described as brisk and improved after recovering from a mysterious incident that caused hair to grow on her face. \nShe is also known for being a key member of the group of friends that include Harry and Ron. \n\nOverall, Hermione is an intelligent and diligent character who values academic achievement and strives to maintain her studies even during challenging circumstances.  It\'s important to note that the context provided is limited and only captures a small portion of Hermione\'s character and behavior within the book "Harry Potter and the Half-Blood Prince." To understand more about Hermione\'s style, behavior, and character, it would be beneficial to explore the entire series and witness her journey and developmen

In [48]:
# Set up our connection to the API.
stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], # API Key reference.
    verbose=True, # Print debug messages.
    engine="stable-diffusion-xl-beta-v2-2-2", # Set the engine to use for generation.
    # Available engines: stable-diffusion-v1 stable-diffusion-v1-5 stable-diffusion-512-v2-0 stable-diffusion-768-v2-0
    # stable-diffusion-512-v2-1 stable-diffusion-768-v2-1 stable-diffusion-xl-beta-v2-2-2 stable-inpainting-v1-0 stable-inpainting-512-v2-0
)


INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443


In [50]:
prompt = """

Hermione is a smart and ambitious student who excels in academics and is passionate about learning. \n\nShe is known for her dedication to studying and completing homework, even while recovering in the hospital wing after a mysterious disappearance. \nHer behavior is described as brisk and improved after recovering from a mysterious incident that caused hair to grow on her face. \nShe is also known for being a key member of the group of friends that include Harry and Ron. \n\nOverall, Hermione is an intelligent and diligent character who values academic achievement and strives to maintain her studies even during challenging circumstances.  It\'s important to note that the context provided is limited and only captures a small portion of Hermione\'s character and behavior within the book "Harry Potter and the Half-Blood Prince." To understand more about Hermione\'s style, behavior, and character, it would be beneficial to explore the entire series and witness her journey and development throughout the storylines.

"""

In [68]:
# Set up our initial generation parameters.
answers = stability_api.generate(
    prompt=prompt, # The prompt to generate from.
    seed=992446762, # If a seed is provided, the resulting generated image will be deterministic.
                    # What this means is that as long as all generation parameters remain the same, you can always recall the same image simply by generating it again.
                    # Note: This isn't quite the case for CLIP Guided generations, which we tackle in the CLIP Guidance documentation.
    steps=40, # Amount of inference steps performed on image generation. Defaults to 30.
    cfg_scale=7.0, # Influences how strongly your generation is guided to match your prompt.
                # Setting this value higher increases the strength in which it tries to match your prompt.
                # Defaults to 7.0 if not specified.
    width=512, # Generation width, defaults to 512 if not included.
    height=512, # Generation height, defaults to 512 if not included.
    samples=1, # Number of images to generate, defaults to 1 if not included.
    sampler=generation.SAMPLER_K_DPMPP_2M # Choose which sampler we want to denoise our generation with.
                                                # Defaults to k_dpmpp_2m if not specified. Clip Guidance only supports ancestral samplers.
                                                # (Available Samplers: ddim, plms, k_euler, k_euler_ancestral, k_heun, k_dpm_2, k_dpm_2_ancestral, k_dpmpp_2s_ancestral, k_lms, k_dpmpp_2m, k_dpmpp_sde)
)

In [69]:
# Set up our warning to print to the console if the adult content classifier is tripped.
# If adult content classifier is not tripped, save generated images.
for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img = Image.open(io.BytesIO(artifact.binary))
            img_path = img.save(str(artifact.seed)+ ".png") # Save our generated images with their seed number as the filename.

INFO:stability_sdk.client:Sending request.
INFO:stability_sdk.client:Got answer d2600b60-e856-4d86-b1a3-96e4d6d46e5f with artifact types ['ARTIFACT_IMAGE', 'ARTIFACT_CLASSIFICATIONS'] in 5.58s


In [70]:
# Display the image
img.show()